In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import os,errno
import sys
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
import pandas as pd
import datetime as dt
from scipy.ndimage.measurements import label
from math import sin, cos, sqrt, atan2, radians
import geopy.distance
%matplotlib inline

dir2='/thorncroftlab_rit/ahenny/rain/'
dir1='/thorncroftlab_rit/ahenny/rain/US/ghcnd_all/'
dir='/thorncroftlab_rit/ahenny/rain/DISSERTATION_SCRIPTS_RESULTS/'

In [ ]:
#read IBTrACS data and convert into simpler format

#ds=xr.open_dataset(dir+'IBTrACS.NA.v04r00.nc')
ds=xr.open_dataset(dir+'IBTrACS.WP.v04r00.nc')
print(ds)
print(ds['track_type'])
#print(ds['mlc_class'])
print(ds['neumann_class'])
print(ds['nature'][185].values.tolist())
print(list(set(ds['nature'].values.flatten().tolist())))
if ds['nature'][100].values.tolist()[0]==b'TS':
    print('YES')

In [ ]:
#First - load dataset and convert times into round numbers of hours (they seem to have .0004..on the end)
#Also replace weird NaN values with np.nan
ds=xr.open_dataset(dir+'IBTrACS.WP.v04r00.nc')
#print(ds)
time=ds.time.values
lat=ds.lat
lon=ds.lon
nature_tc=ds.nature.values
years=np.zeros((ds.storm.size,ds.date_time.size))
months=np.zeros((ds.storm.size,ds.date_time.size))
days=np.zeros((ds.storm.size,ds.date_time.size))
hours=np.zeros((ds.storm.size,ds.date_time.size))
nature=np.zeros((ds.storm.size,ds.date_time.size))
for i in range(ds.storm.size):
    print(i)
    for j in range(ds.date_time.size):
        time_single=time[i,j]
        time_single=pd.to_datetime(time_single)
        year=time_single.year
        month=time_single.month
        day=time_single.day
        hour=time_single.hour
        nature_value=nature_tc[i,j]
        if year>0 and month>0 and day>0 and hour>=0:
            years[i,j]=year
            months[i,j]=month
            days[i,j]=day
            hours[i,j]=hour
            if nature_value==b'TS':
                nature[i,j]=1
            else:
                nature[i,j]=0
        else:
            years[i,j]=np.nan
            months[i,j]=np.nan
            days[i,j]=np.nan
            hours[i,j]=np.nan
            nature[i,j]=np.nan

dk=xr.Dataset()
dk['years']=(('storm','date_time'),years)
dk['months']=(('storm','date_time'),months)
dk['days']=(('storm','date_time'),days)
dk['hours']=(('storm','date_time'),hours)
dk['nature']=(('storm','date_time'),nature)
dk.coords['storm']=ds.storm
dk.coords['date_time']=ds.date_time

In [ ]:
print(np.amin(days))
print(np.amax(hours))
print(dk.hours)

In [ ]:
try:
    os.remove(dir+'ibtracs_wp_time.nc')
except OSError:
    pass
dk.to_netcdf(dir+'ibtracs_wp_time.nc',mode='w',format='NETCDF4')

In [ ]:
ds1=xr.open_dataset(dir+'ibtracs_wp_time.nc')
years=ds1.years
months=ds1.months
days=ds1.days
hours=ds1.hours

time_select=dt.datetime(1996,10,21,18)
#b=lat.sel(time==time_select)
print(time_select)
a=lat.where(years==1996,drop=True)
a=a.where(months==10,drop=True)
a=a.where(days==21,drop=True)
a=a.where(hours==18,drop=True)

print(a)
